In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
from numpy.random import random

In [2]:
torch.__version__

'2.1.0+cu121'

In [3]:
x = random((30,2))
y = np.dot(x,[2.,-3.]) +1.
w_source = np.array([2.,-3.])
b_source = np.array([1.])

In [4]:
x[:5]

array([[0.03993043, 0.96609203],
       [0.37884645, 0.87867106],
       [0.89296698, 0.77253695],
       [0.47642663, 0.19304256],
       [0.05444117, 0.63777982]])

In [5]:
from mpl_toolkits.mplot3d import Axes3D
def plot_figs(fig_num,elev,azim,x,y,weights,bias):
  fig = plt.figure(fig_num,figsize=(4,3))
  plt.clf()
  ax = Axes3D(fig,elev=elev,azim=azim)
  ax.scatter(x[:,0],x[:,1],y)
  ax.plot_surface(np.array([[0, 0], [1, 1]]),
   np.array([[0, 1], [0, 1]]),
   (np.dot(np.array([[0, 0, 1, 1],
   [0, 1, 0, 1]]).T, weights) + bias).reshape((2, 2)),alpha=.5)
  ax.set_xlabel('x_1')
  ax.set_ylabel('x_2')
  ax.set_zlabel('y')

def plot_views(x,y,w,b):
  elev = 43.5
  azim = -110
  plot_figs(1,elev,azim,x,y,w,b[0])
  plt.show()

In [6]:
plot_views(x,y,w_source,b_source)

<Figure size 400x300 with 0 Axes>

In [7]:
w_init = random(2)
b_init = random(1)
w = w_init
b = b_init
print('Initial values of the params',w,b)


Initial values of the params [0.92316734 0.38644598] [0.73824242]


In [8]:
def forward(x):
  return x.dot(w)+b
def loss(x,y):
  y_pred = forward(x)
  return (y-y_pred)**2

print('Initial Loss:',np.sum([loss(x_val,y_val) for x_val, y_val in zip(x,y)]))

def gradient(x,y):
  return 2*(x.dot(w)+b-y)*x,2*(x.dot(w)+b - y)

lr = 1e-2

for epoch in range(10):
  grad_w = np.array([0,0])
  grad_b = np.array([0])
  l = 0
  for x_val,y_val in zip(x,y):
    grad_w = np.add(grad_w,gradient(x_val,y_val)[0])
    grad_b = np.add(grad_b,gradient(x_val,y_val)[1])
    l+=loss(x_val,y_val)
  w = w - lr * grad_w
  b = b - lr * grad_b
  print('progress:','epoch:',epoch,'loss',l[0])

print('Estimation of the params',w,b)




Initial Loss: 68.98904961765692
progress: epoch: 0 loss 68.98904961765692
progress: epoch: 1 loss 19.34984310403876
progress: epoch: 2 loss 17.933526166905704
progress: epoch: 3 loss 16.625258665006264
progress: epoch: 4 loss 15.416520412530101
progress: epoch: 5 loss 14.299489940717352
progress: epoch: 6 loss 13.266975030967423
progress: epoch: 7 loss 12.312361263416742
progress: epoch: 8 loss 11.42956481827919
progress: epoch: 9 loss 10.612989175938411
Estimation of the params [ 1.12020646 -0.92426687] [0.1765857]


In [9]:
plot_views(x,y,w,b)

<Figure size 400x300 with 0 Axes>

In [10]:
dtype = torch.FloatTensor

In [11]:
x_t = torch.from_numpy(x).type(dtype)
y_t = torch.from_numpy(y).type(dtype).unsqueeze(1)

In [13]:
w_init_t = torch.from_numpy(w_init).type(dtype)
b_init_t = torch.from_numpy(b_init).type(dtype)

w_t = w_init_t.clone()
w_t.unsqueeze_(1)
b_t = b_init_t.clone()
b_t.unsqueeze_(1)
print('initial values of the params:',w_t,b_t)

initial values of the params: tensor([[0.9232],
        [0.3864]]) tensor([[0.7382]])


In [16]:
def forward_t(x):
  return x.mm(w_t)+b_t

def loss_t(x,y):
  y_pred = x.mm(w_t)+b_t
  return (y_pred-y).pow(2).sum()

def gradient_t(x,y):
  return 2*torch.mm(torch.t(x),x.mm(w_t)+b_t-y),2*(x.mm(w_t)+b_t-y).sum()

lr = 1e-2
for epoch in range(10):
  l_t = loss_t(x_t,y_t)
  grad_w,grad_b = gradient_t(x_t,y_t)
  w_t = w_t - lr*grad_w
  b_t = b_t - lr*grad_b
  print('Progress:','Epoch:',epoch,'Loss',l_t.item())

print('Estimation of the params',w,b)



Progress: Epoch: 0 Loss 4.783901214599609
Progress: Epoch: 1 Loss 4.457211494445801
Progress: Epoch: 2 Loss 4.154014587402344
Progress: Epoch: 3 Loss 3.8725438117980957
Progress: Epoch: 4 Loss 3.6111700534820557
Progress: Epoch: 5 Loss 3.368391275405884
Progress: Epoch: 6 Loss 3.142822265625
Progress: Epoch: 7 Loss 2.9331836700439453
Progress: Epoch: 8 Loss 2.738295078277588
Progress: Epoch: 9 Loss 2.5570693016052246
Estimation of the params [ 1.12020646 -0.92426687] [0.1765857]


Linear Regression with Autograd

In [17]:
w_v = w_init_t.clone().unsqueeze(1)
w_v.requires_grad_(True)
b_v  = b_init_t.clone().unsqueeze(1)
b_v.requires_grad_(True)
print('Initial values of the params',w_v.data,b_v.data)

Initial values of the params tensor([[0.9232],
        [0.3864]]) tensor([[0.7382]])


In [18]:
for epoch in range(10):
  y_pred = x_t.mm(w_v)+b_v
  loss = (y_pred - y_t).pow(2).sum()
  loss.backward()

  with torch.no_grad():
    w_v -= lr * w_v.grad
    b_v -= lr* b_v.grad

  w_v.grad.zero_()
  b_v.grad.zero_()

  print('Progress:','Epoch:',epoch,'loss',loss.data.item())

print("estimation of the parameters:", w_v.data, b_v.data.t() )


Progress: Epoch: 0 loss 68.98905181884766
Progress: Epoch: 1 loss 19.349842071533203
Progress: Epoch: 2 loss 17.93352699279785
Progress: Epoch: 3 loss 16.625259399414062
Progress: Epoch: 4 loss 15.416521072387695
Progress: Epoch: 5 loss 14.299488067626953
Progress: Epoch: 6 loss 13.266973495483398
Progress: Epoch: 7 loss 12.312360763549805
Progress: Epoch: 8 loss 11.429563522338867
Progress: Epoch: 9 loss 10.612988471984863
estimation of the parameters: tensor([[ 1.1202],
        [-0.9243]]) tensor([[0.1766]])


Linear Regression with Neural Network

In [25]:
model = torch.nn.Sequential(torch.nn.Linear(2,1),)
for m in model.children():
  m.weight.data = w_init_t.clone().unsqueeze(0)
  m.bias.data = b_init_t.clone()

loss_fn = torch.nn.MSELoss(reduction='sum')
model.train()
for epoch in range(10):
  y_pred = model(x_t)
  loss = loss_fn(y_pred,y_t)
  model.zero_grad()
  loss.backward()
  with torch.no_grad():
    for param in model.parameters():
      param.data -= lr*param.grad
  print('Progress','Epoch',epoch,'Loss',loss.data.item())

print("Estimation of the params:")
for param in model.parameters():
  print(param)

Progress Epoch 0 Loss 68.98905181884766
Progress Epoch 1 Loss 19.349842071533203
Progress Epoch 2 Loss 17.93352699279785
Progress Epoch 3 Loss 16.625259399414062
Progress Epoch 4 Loss 15.416521072387695
Progress Epoch 5 Loss 14.299488067626953
Progress Epoch 6 Loss 13.266973495483398
Progress Epoch 7 Loss 12.312360763549805
Progress Epoch 8 Loss 11.429563522338867
Progress Epoch 9 Loss 10.612988471984863
Estimation of the params:
Parameter containing:
tensor([[ 1.1202, -0.9243]], requires_grad=True)
Parameter containing:
tensor([0.1766], requires_grad=True)


Using Optim package to update the weights and bias

In [26]:
model = torch.nn.Sequential(torch.nn.Linear(2,1),)
for m in model.children():
  m.weight.data = w_init_t.clone().unsqueeze(0)
  m.bias.data = b_init_t.clone()

loss_fn = torch.nn.MSELoss(reduction='sum')
model.train()
optimizer = torch.optim.SGD(model.parameters(),lr=lr)

for epoch in range(10):
  y_pred = model(x_t)
  loss = loss_fn(y_pred,y_t)
  print('Progress:','Epoch:',epoch,'Loss',loss.item())
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print('Estimation of the params')
for param in model.parameters():
  print(param)




Progress: Epoch: 0 Loss 68.98905181884766
Progress: Epoch: 1 Loss 19.349842071533203
Progress: Epoch: 2 Loss 17.93352699279785
Progress: Epoch: 3 Loss 16.625259399414062
Progress: Epoch: 4 Loss 15.416521072387695
Progress: Epoch: 5 Loss 14.299489974975586
Progress: Epoch: 6 Loss 13.266973495483398
Progress: Epoch: 7 Loss 12.312360763549805
Progress: Epoch: 8 Loss 11.429563522338867
Progress: Epoch: 9 Loss 10.612988471984863
Estimation of the params
Parameter containing:
tensor([[ 1.1202, -0.9243]], requires_grad=True)
Parameter containing:
tensor([0.1766], requires_grad=True)


Solving Linear Regression algebrically

In [27]:
xb_t = torch.cat((x_t,torch.ones(30).unsqueeze(1)),1)
sol = torch.linalg.lstsq(xb_t,y_t)
sol.solution

tensor([[ 2.0000],
        [-3.0000],
        [ 1.0000]])

Exercise

In [28]:
x = random((300,2))
y = np.dot(x,[2.,-3.]) + 1
x_t = torch.from_numpy(x).type(dtype)
y_t = torch.from_numpy(y).type(dtype).unsqueeze(1)

In [31]:
lr = 0.001

In [34]:
model = torch.nn.Sequential(torch.nn.Linear(2,1),)
for m in model.children():
  m.weight.data = w_init_t.clone().unsqueeze(0)
  m.bias.data = b_init_t.clone()

loss_fn = torch.nn.MSELoss(reduction = 'sum')

model.train()

optimizer = torch.optim.SGD(model.parameters(), lr=lr)


for epoch in range(100):
    y_pred = model(x_t)
    loss = loss_fn(y_pred, y_t)
    print("progress:", "epoch:", epoch, "loss",loss.item())
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


# After training
print("estimation of the parameters:")
for param in model.parameters():
    print(param)

progress: epoch: 0 loss 606.6519775390625
progress: epoch: 1 loss 249.57183837890625
progress: epoch: 2 loss 225.75772094726562
progress: epoch: 3 loss 205.57424926757812
progress: epoch: 4 loss 187.21250915527344
progress: epoch: 5 loss 170.5006103515625
progress: epoch: 6 loss 155.28952026367188
progress: epoch: 7 loss 141.44383239746094
progress: epoch: 8 loss 128.84031677246094
progress: epoch: 9 loss 117.36698913574219
progress: epoch: 10 loss 106.92195129394531
progress: epoch: 11 loss 97.41255187988281
progress: epoch: 12 loss 88.75453186035156
progress: epoch: 13 loss 80.87122344970703
progress: epoch: 14 loss 73.69293212890625
progress: epoch: 15 loss 67.15625
progress: epoch: 16 loss 61.20348358154297
progress: epoch: 17 loss 55.782161712646484
progress: epoch: 18 loss 50.84455871582031
progress: epoch: 19 loss 46.34724426269531
progress: epoch: 20 loss 42.25072479248047
progress: epoch: 21 loss 38.519046783447266
progress: epoch: 22 loss 35.119510650634766
progress: epoch: 2